<a href="https://colab.research.google.com/github/NinahEulalie/GoogleCollab_projects/blob/main/CLAUDE_Systeme_Recommandation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UTILISATION DE BERT

In [14]:
!pip install scikit-learn

In [1]:
!pip install transformers

In [2]:
from google.colab import files

# Upload du fichier pour les offres d'emploi
uploaded = files.upload()

# Get the file name and assure that the correct filename is captured
path_to_poste = next(iter(uploaded))

Saving cleaned_[OK]_offres-emploi-stage-province-sud (1).csv to cleaned_[OK]_offres-emploi-stage-province-sud (1).csv


In [3]:
from google.colab import files

# Upload du fichier pour les CVs
uploaded = files.upload()

# Get the file name
path_to_cv = next(iter(uploaded))

Saving cleaned_900_cv_dataset_francais.csv to cleaned_900_cv_dataset_francais.csv


In [4]:
#import pandas as pd

# Chargement du fichier en DataFrame (df)
#postes_df = pd.read_csv(path_to_poste, encoding='utf-8')

# Afficher les 5 premières lignes
#print(postes_df.head())

import pandas as pd

# Lecture du fichier CSV en tenant compte des champs avec guillemets et des caractères spéciaux
postes_df = pd.read_csv(
    path_to_poste,
    delimiter=",",
    encoding="utf-8",
    quotechar='"',
    on_bad_lines='skip'  # Skip lines with errors in parsing
)

# Afficher les 5 premières lignes pour vérifier
print(postes_df.head())


                                 id       reference  \
0  8a81868591b661e90192da731a987c0f  of-2024-10-337   
1  8a81868591b661e90192d5a4ff990262  of-2024-10-329   
2  8a81868591b661e90192d5368d8b6556  of-2024-10-325   
3  8a81868591b661e90192d13ad5a52bf8  of-2024-10-318   
4  8a81868591b661e90192c25016ae1412  of-2024-10-303   

                                               title        prisePoste  \
0       technicien polyvalent (h/f) (of-2024-10-337)  dès que possible   
1             attache(e) banqueting (of-2024-10-329)  dès que possible   
2      développeur php sénior (h/f) (of-2024-10-325)  dès que possible   
3  manipulateur / manipulatrice en radiothérapie ...  dès que possible   
4                         jardinier (of-2024-10-303)  dès que possible   

         niveauDiplome  commune  \
0  cap, bep, dnb, bepc      NaN   
1                bac+2      NaN   
2                bac+5      NaN   
3                bac+2      NaN   
4                bac+2      NaN   

             

In [5]:
import pandas as pd

# Chargement du fichier en DataFrame (df)
cv_df = pd.read_csv(path_to_cv, encoding='utf-8')

# Afficher les 5 premières lignes
print(cv_df.head())

  Sexe  Prénom        Nom                      Email             Téléphone  \
0    f  céline  bourgeois  céline.bourgeois@live.com  +33 (0)1 42 76 26 00   
1    m  roland    chauvin     roland.chauvin@noos.fr            0373253850   
2    m  thomas    guillon     thomas.guillon@noos.fr        02 34 95 31 66   
3    m  pierre     seguin     pierre.seguin@tele2.fr        02 90 44 88 55   
4    f  élodie     seguin    élodie.seguin@orange.fr  +33 (0)5 86 52 96 91   

              Ville        Domaine Niveau d'expérience  \
0   martins-sur-mer   météorologie            débutant   
1             barbe         design       intermédiaire   
2           bonneau       economie              senior   
3  delmas-les-bains  communication              expert   
4           noëldan   météorologie              junior   

                                       Formation  \
0  école d'ingénieur - loiseau leconte sa (2023)   
1             licence - meyer leroux sarl (2013)   
2            licence - dum

In [17]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split  # Ajout de cette ligne
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
from tensorflow.keras import layers, models
import random
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

class CVMatchingSystem:
    def __init__(self, bert_model_name="bert-base-multilingual-cased", max_length=128):
        print("Initialisation du système...")
        self.tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.bert = TFBertModel.from_pretrained(bert_model_name)
        self.max_length = max_length
        print("Initialisation terminée!")

    def create_bert_embeddings(self, texts):
        print(f"\nCréation des embeddings BERT pour {len(texts)} textes...")

        # Tokenization avec attention mask
        print("Tokenization en cours...")
        encoded = self.tokenizer(
            texts,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='tf'
        )

        # Obtenir les embeddings BERT
        print("Génération des embeddings...")
        outputs = self.bert(encoded['input_ids'], attention_mask=encoded['attention_mask'])
        embeddings = outputs.last_hidden_state[:, 0, :]
        print(f"Embeddings générés avec shape: {embeddings.shape}")
        return embeddings

    def generate_training_pairs(self, postes_df, cv_df, n_positive=1000, n_negative=1000):
        print("\nGénération des paires d'entraînement...")
        print(f"Nombre de postes: {len(postes_df)}")
        print(f"Nombre de CVs: {len(cv_df)}")

        # Générer les embeddings
        print("\nPréparation des textes...")
        postes_text = (postes_df['description'] + " " + postes_df['niveauExperience'] + " " + postes_df['domaineActivite']).tolist()
        cv_text = (cv_df['Domaine'] + " " + cv_df['Expériences'] + " " + cv_df['Compétences']).tolist()

        postes_embeddings = self.create_bert_embeddings(postes_text)
        cv_embeddings = self.create_bert_embeddings(cv_text)

        # Calculer la matrice de similarité
        print("\nCalcul de la matrice de similarité...")
        similarity_matrix = cosine_similarity(postes_embeddings, cv_embeddings)
        print(f"Shape de la matrice de similarité: {similarity_matrix.shape}")

        # Générer des paires positives
        print("\nGénération des paires positives...")
        positive_pairs = []
        for i in tqdm(range(len(postes_df))):
            best_matches = np.argsort(similarity_matrix[i])[-5:]
            for cv_idx in best_matches:
                if similarity_matrix[i][cv_idx] > 0.7:
                    positive_pairs.append((i, cv_idx, 1))

        # Générer des paires négatives
        print("\nGénération des paires négatives...")
        negative_pairs = []
        for i in tqdm(range(len(postes_df))):
            worst_matches = np.argsort(similarity_matrix[i])[:5]
            for cv_idx in worst_matches:
                if similarity_matrix[i][cv_idx] < 0.3:
                    negative_pairs.append((i, cv_idx, 0))

        print(f"\nNombre de paires positives trouvées: {len(positive_pairs)}")
        print(f"Nombre de paires négatives trouvées: {len(negative_pairs)}")

        # Équilibrer et mélanger les données
        positive_pairs = random.sample(positive_pairs, min(len(positive_pairs), n_positive))
        negative_pairs = random.sample(negative_pairs, min(len(negative_pairs), n_negative))
        all_pairs = positive_pairs + negative_pairs
        random.shuffle(all_pairs)

        print(f"\nNombre final de paires d'entraînement: {len(all_pairs)}")
        print(f"Distribution des labels: {sum(p[2] for p in all_pairs)} positifs, {len(all_pairs) - sum(p[2] for p in all_pairs)} négatifs")

        return all_pairs

    def build_model(self, vocab_size, embedding_dim=64, hidden_units=32):
        print("\nConstruction du modèle...")

        # Modèle Siamois avec attention
        input_poste = layers.Input(shape=(self.max_length,))
        input_cv = layers.Input(shape=(self.max_length,))

        # Couche d'embedding partagée
        embedding_layer = layers.Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            mask_zero=True
        )

        # Attention layer
        attention = layers.MultiHeadAttention(
            num_heads=4,
            key_dim=embedding_dim
        )

        # Traitement des postes
        x_poste = embedding_layer(input_poste)
        x_poste = attention(x_poste, x_poste)
        x_poste = layers.Bidirectional(layers.LSTM(hidden_units))(x_poste)
        x_poste = layers.Dropout(0.3)(x_poste)

        # Traitement des CVs
        x_cv = embedding_layer(input_cv)
        x_cv = attention(x_cv, x_cv)
        x_cv = layers.Bidirectional(layers.LSTM(hidden_units))(x_cv)
        x_cv = layers.Dropout(0.3)(x_cv)

        # Calcul de similarité avec couches supplémentaires
        merged = layers.Concatenate()([x_poste, x_cv])
        merged = layers.Dense(64, activation='relu')(merged)
        merged = layers.Dropout(0.3)(merged)
        merged = layers.Dense(32, activation='relu')(merged)
        output = layers.Dense(1, activation='sigmoid')(merged)

        model = models.Model(inputs=[input_poste, input_cv], outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss='binary_crossentropy',
            metrics=['accuracy', tf.keras.metrics.AUC()]
        )

        print("\nArchitecture du modèle:")
        model.summary()

        return model

    tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
    tokenizer.fit_on_texts(postes_text + CV_text)  # Combinez tous les textes pour construire le vocabulaire

    # Conversion en séquences numériques
    X_train_postes = tokenizer.texts_to_sequences(postes_text)
    X_train_cvs = tokenizer.texts_to_sequences(CV_text)

    # Appliquez un padding pour uniformiser les longueurs
    max_length = 128
    X_train_postes = pad_sequences(X_train_postes, maxlen=max_length, padding='post')
    X_train_cvs = pad_sequences(X_train_cvs, maxlen=max_length, padding='post')

    # Assurez-vous que y_train est sous forme de tableau numpy si ce n'est pas déjà fait
    import numpy as np
    y_train = np.array(y_train)

    # Vérifiez les dimensions pour éviter les erreurs
    print("\nVérification des dimensions après le prétraitement...")
    print(f"X_train_postes: {X_train_postes.shape}")
    print(f"X_train_cvs: {X_train_cvs.shape}")
    print(f"y_train: {y_train.shape}")

    def train(self, X_train_postes, X_train_cvs, y_train, epochs=40, batch_size=32):
        print("\nPréparation de l'entraînement...")
        print(f"Données d'entraînement:")
        print(f"X_train_postes shape: {X_train_postes.shape}")
        print(f"X_train_cvs shape: {X_train_cvs.shape}")
        print(f"y_train shape: {y_train.shape}")

        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True,
                verbose=1
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=3,
                verbose=1
            ),
            tf.keras.callbacks.ModelCheckpoint(
                'best_model.keras',
                monitor='val_accuracy',
                save_best_only=True,
                verbose=1
            )
        ]

        model = self.build_model(self.tokenizer.vocab_size)

        print("\nDébut de l'entraînement...")
        history = model.fit(
            [X_train_postes, X_train_cvs],
            y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            callbacks=callbacks,
            verbose=1  # Affiche la barre de progression
        )

        # Affichage des métriques finales
        print("\nRésultats finaux:")
        for metric in history.history.keys():
            print(f"{metric}: {history.history[metric][-1]:.4f}")

        return model, history

# Exemple d'utilisation:
print("=== Début du programme ===")
print("\nChargement des données...")
# Supposons que postes_df et cv_df sont déjà chargés
matching_system = CVMatchingSystem()

print("\nGénération des paires d'entraînement...")
training_pairs = matching_system.generate_training_pairs(postes_df, cv_df)

print("\nPréparation des données pour l'entraînement...")
X_postes, X_cvs, y = zip(*training_pairs)
X_train_postes, X_test_postes, X_train_cvs, X_test_cvs, y_train, y_test = train_test_split(
    np.array(X_postes), np.array(X_cvs), np.array(y), test_size=0.2, random_state=42
)

print("\nDébut de l'entraînement du modèle...")
model, history = matching_system.train(X_train_postes, X_train_cvs, y_train)

print("\nÉvaluation sur le jeu de test...")
test_results = model.evaluate([X_test_postes, X_test_cvs], y_test, verbose=1)
print(f"\nRésultats sur le jeu de test:")
for metric, value in zip(model.metrics_names, test_results):
    print(f"{metric}: {value:.4f}")

print("\n=== Fin du programme ===")

=== Début du programme ===

Chargement des données...
Initialisation du système...


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Initialisation terminée!

Génération des paires d'entraînement...

Génération des paires d'entraînement...
Nombre de postes: 308
Nombre de CVs: 900

Préparation des textes...

Création des embeddings BERT pour 308 textes...
Tokenization en cours...
Génération des embeddings...
Embeddings générés avec shape: (308, 768)

Création des embeddings BERT pour 900 textes...
Tokenization en cours...
Génération des embeddings...
Embeddings générés avec shape: (900, 768)

Calcul de la matrice de similarité...
Shape de la matrice de similarité: (308, 900)

Génération des paires positives...


100%|██████████| 308/308 [00:00<00:00, 10288.18it/s]



Génération des paires négatives...


...

 Total params: 7,777,409 (29.67 MB)

 Trainable params: 7,777,409 (29.67 MB)

 Non-trainable params: 0 (0.00 B)


Début de l'entraînement...
Epoch 1/40


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("functional_1/Cast:0", shape=(32,), dtype=float32). Expected shape (None, 128), but input has incompatible shape (32,)[0m

Arguments received by Functional.call():
  • inputs=('tf.Tensor(shape=(32,), dtype=int64)', 'tf.Tensor(shape=(32,), dtype=int64)')
  • training=True
  • mask=('None', 'None')